In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score
import lightgbm as lgb

PA = pd.read_csv("data.csv")
X = PA[["AGE", "SEX", "BMI", "DURATION", "DDD",  "PAC"]]
y = PA["Clinical_outcome"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
skf = StratifiedKFold(n_splits=5, shuffle=True)
LGBM = lgb.LGBMClassifier()
param_grid = {"max_depth": [2, 3, 5, 7, 10],
              "learning_rate" : [0.001,0.01,0.05,0.1],
              "num_leaves": [1,3,5,7,10],
              "min_data_in_leaf": [0, 3, 5, 7, 10],
              "n_estimators": [50,100,300,500,700,1000]
             }
grid_result = GridSearchCV(estimator = LGBM,
                           param_grid = param_grid,
                           cv = skf,
                           n_jobs = -1)
grid_result.fit(X_train, y_train)
print(grid_result.score(X_test, y_test))

In [ ]:
y_pred = grid_result.predict(X_test)
confusion_matrix(y_test, y_pred, labels=[1,0])

In [ ]:
prob = grid_result.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds= roc_curve(y_test, prob)
fig = plt.figure(figsize=(5,5))
plt.plot(fpr, tpr)
plt.xlabel("1 - Specificity", fontsize=10)
plt.ylabel("Sensitivity", fontsize=10)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
roc_auc_score(y_test, prob)